## Preparation

In [2]:
# Code reference: https://huggingface.co/docs/transformers/tasks/language_modeling

# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

! pip install transformers[torch]
! pip install accelerate -U
! pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [1]:
import os
import json

from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from transformers import DataCollatorForLanguageModeling, EarlyStoppingCallback

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# file_path = 'drive/MyDrive/SNLP/emotion_alpaca_v4_cleaned.json'
file_path = 'alpaca-instruction/emotion_alpaca_v4_cleaned.json'

with open(file_path, 'r') as file:
    data = json.load(file)

In [3]:
dataset = Dataset.from_dict({
    'instruction': [item['instruction'] for item in data],
    'input': [item['input'] for item in data],
    'output': [item['rewritten_output'] for item in data]
})

In [4]:
dataset = dataset.train_test_split(test_size=0.1)

In [5]:
dataset['train'][0]

{'instruction': 'Grade the following essay with a score between 1-10.',
 'input': 'The environment is something we all rely on, yet many take it for granted. The rapid population growth is leading to an unsustainable number of resources. Climate change is a major concern, with many creatures becoming extinct due to the effects of global warming. We need to start making more of an effort now to protect our environment.',
 'output': 'Great job! Your revised output effectively communicates your approval for the content and provides constructive criticism. It accurately captures the essence of the input and highlights specific areas where improvement can be made. Keep up the good work!'}

In [20]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [21]:
def formatting_func(batch):
    # Process each example in the batch and return a list of formatted strings
    return [
        f"{tokenizer.bos_token} Instruction: {instr} Input: {inp} Output: {out} {tokenizer.eos_token}"
        for instr, inp, out in zip(batch['instruction'], batch['input'], batch['output'])
    ]

In [22]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [24]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [26]:
LR = 5e-5            # Learning rate
PATIENCE = 10        # Patience for early stopping
BSZ = 4              # Batch size
EVAL_EVERY = 200     # Evaluate every X steps
SAVE_EVERY = 200     # Save model checkpoint every X steps
MAX_EPOCHS = 10      # Maximum number of epochs

In [27]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="gpt2_finetuned_instruction",
    evaluation_strategy="steps",
    learning_rate=LR,
    per_device_train_batch_size=BSZ,
    per_device_eval_batch_size=BSZ,
    num_train_epochs=MAX_EPOCHS,
    eval_steps=EVAL_EVERY,
    save_steps=SAVE_EVERY,
    load_best_model_at_end=True,
    gradient_accumulation_steps=2,
    fp16=True
)

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
    max_seq_length=1024,
    formatting_func=formatting_func
)

Map:   0%|          | 0/18798 [00:00<?, ? examples/s]

Map:   0%|          | 0/2089 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB. GPU 0 has a total capacty of 15.68 GiB of which 58.25 MiB is free. Including non-PyTorch memory, this process has 15.37 GiB memory in use. Of the allocated memory 14.89 GiB is allocated by PyTorch, and 175.58 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
# Start training
trainer.train()

  0%|          | 0/93990 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 15.68 GiB of which 12.75 MiB is free. Including non-PyTorch memory, this process has 15.37 GiB memory in use. Of the allocated memory 14.89 GiB is allocated by PyTorch, and 175.33 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.save_pretrained("./model_folder")
tokenizer.save_pretrained("./model_folder")

In [ ]:
eval_results = trainer.evaluate(eval_dataset=dataset['test'])
print(eval_results)

In [ ]:
from transformers import pipeline

# Load the trained model into a pipeline for easy inference
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example prompt
prompt = "The capital of France is"
generated = pipe(prompt, max_length=50, num_return_sequences=1)

print("Input Prompt:", prompt)
print("Generated Text:", generated[0]["generated_text"])